In [1]:
from Bio import AlignIO, SeqIO
from Bio.Seq import Seq
import os
import glob
import pandas as pd

In [2]:
DIR = "/home/yutianc/sars-cov-2-dms-ref"
DATA_DIR = "/home/yutianc/sars-cov-2-dms-ref/data"

In [3]:
RBD_COORDS = [22553, 23155] # 1-based idx

records = AlignIO.read(os.path.join(DIR, "output/escape_all.fa"), format="fasta")

for rec in records:
    seq_id = rec.id.split("_")[-2].lower()

    mut_file = glob.glob(os.path.join(DATA_DIR, seq_id, "mut1*.csv"))[0]
    mut_df = pd.read_csv(mut_file)
    mut_wt = list(mut_df["wt"])
    wt = []
    coords = []
    start = 331

    for pos in range(22553-1, 23155, 3):
        try:
            aa = str(Seq(rec.seq[pos: pos+3]).translate())
            wt.append(aa)
            coords.append(start) # return 1-based idx
            start += 1
        except:
            start += 1
            continue

            
    # df = pd.DataFrame({"wt": wt, "pos": coords})
    df = pd.DataFrame({"wt": wt, "mut_wt": mut_wt, "coords": coords})
    df_diff = df.loc[df["wt"] != df["mut_wt"], :]
    print(df_diff)


FileNotFoundError: [Errno 2] No such file or directory: '/home/yutianc/sars-cov-2-dms-ref/output/escape_all.fa'

In [ ]:
RBD_COORDS = [22553, 23155] # 1-based idx

rec = SeqIO.read("/home/yutianc/bjorn_rep/data/sc2/NC_045512.2_BA.1.fasta", format="fasta")

mut_file = "/home/yutianc/sars-cov-2-dms-ref/data/ba.1/mut1_BA.1_Omicron_baseline_EPI_ISL_10000028.csv"
mut_df = pd.read_csv(mut_file)
mut_wt = list(mut_df["wt"])

wt = []
coords = []
start = 331

for pos in range(22553-1, 23155, 3):
    try:
        aa = str(Seq(rec.seq[pos: pos+3]).translate())
        wt.append(aa)
        coords.append(start) # return 1-based idx
        start += 1
    except:
        start += 1
        continue


df = pd.DataFrame({"wt": wt, "mut_wt": mut_wt, "coords": coords})
df_diff = df.loc[df["wt"] != df["mut_wt"], :]
print(df_diff)


In [ ]:
from pathlib import Path

FA_EXTS = {".fa", ".fasta", ".fna"}

def iter_fasta_files(root: str):
    root_path = Path(root)
    for p in root_path.rglob("*"):
        if p.is_file() and p.suffix.lower() in FA_EXTS:
            yield p


def extract_ids_from_fasta_file(fp: Path):
    ids = []
    fname = []
    with fp.open("r", encoding="utf-8", errors="replace") as f:
        for line in f:
            if line.startswith(">"):
                # first token after '>' (drops description)
                seq_id = line[1:].strip().split()[0]
                if seq_id:
                    try:
                        sra = seq_id.split('/')[2]
                        fa_name = (f.name.split("/")[-1]).split('.')[0]
                        if sra != fa_name:
                            ids.append(sra)
                            fname.append(fa_name)
                    except:
                        continue
    return ids, fname

def extract_ids_from_dir(root: str):
    out_id, out_fname = [], []
    for fp in iter_fasta_files(root):
        ids, fname = extract_ids_from_fasta_file(fp)
        out_id.extend(ids)
        out_fname.extend(fname)
    print(len(out_fname))
    print(len(out_id))

    return pd.DataFrame({"ID": out_id, "fname": out_fname})

if __name__ == "__main__":
    root_dir = "/home/yutianc/bjorn_rep/data/sc2/consensus_sequences/"
    df = extract_ids_from_dir(root_dir)



In [ ]:
d = dict(zip(df["fname"], df["ID"]))


In [ ]:
df

In [ ]:
lineage_all = pd.read_csv("/home/yutianc/bjorn_rep/data/sc2/lineage_report.csv")
lineage_all["taxon"] = lineage_all["taxon"].map(d).fillna(lineage_all["taxon"])
lineage_all = lineage_all[["taxon", "lineage"]]
lineage_all = lineage_all.rename(columns={"taxon": "sample", "lineage": "Genotype"})
lineage_all.to_csv("/home/yutianc/bjorn_rep/data/sc2/lineage_report_cleaned.tsv", sep='\t', index=False)


In [ ]:
ls

In [ ]:
root  = Path('/home/yutianc/bjorn_rep/data/sc2/consensus_sequences/')
sra_all = [(f.name).split(".")[0] for f in root.rglob("*") if f.is_file()]

In [ ]:
sra_all

In [ ]:
meta = pd.read_csv("/home/jmcfeeters/sc2_data/2025.06.12_outputs_sc2/metadata/normalized_metadata.tsv", sep='\t')
meta = meta[:len(sra_all)]
meta["Accession"] = sra_all
len(meta)

In [ ]:
meta.tail()

In [ ]:
meta.to_csv('/home/yutianc/muninn/inputs/metadata_sc2_all.tsv', sep='\t', index=False)

In [ ]:
mut = pd.read_csv("/home/yutianc/muninn/inputs/escape_all_1000/mutations.tsv", sep='\t')
meta_sample = meta[meta["Accession"].isin(list(mut["sra"].unique()))]
meta_sample

In [ ]:
mut["sra"].nunique()